In [ ]:
%pip install unidecode

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import month,concat_ws,col,when,to_timestamp,to_date,date_format,desc,lit,year,count,year,upper,initcap,regexp_extract
import pandas as pd
import numpy as np

# Crear una SparkSession
spark = SparkSession.builder.appName("AzureSQL").getOrCreate()

## Credenciales BD
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Credenciales.txt"
keys = pd.read_csv(inpath, sep = ',')
display(keys)

# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value

# Configurar las propiedades de la conexión
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database_datamart}"
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

# Conexion al Blob
spark.conf.set(clave_blob, access_key_blob)

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re
from unidecode import unidecode

####  FUNCIONES  ####

# Limpieza de texto
def limpiar_texto(texto):
    if texto is None:
        return 'Sin Registro'
    
    # Mayuscula
    texto_mayus = str.upper(texto)

    # Eliminar valores numéricos utilizando expresiones regulares
    texto_sin_numeros = re.sub(r'\d+', '', str(texto_mayus))

    # Remover tildes y caracteres especiales de entonacion
    texto_sin_tildes = unidecode(texto_sin_numeros)
    #texto_sin_tildes = unidecode(texto)

    # Quitar caracteres no alfabéticos y no espacios en blanco
    texto_limpio = re.sub(r'[^a-zA-Z\s]', '', texto_sin_tildes)

    # Aplicar trim para eliminar espacios en blanco al principio y al final
    texto_limpio = texto_limpio.strip()
    
    return texto_limpio


# Funcion like ampliada
def asignar_ciudad(row):
    # el diccionario se usa para buscar una coincidencia parcial entre las claves y los valores en la columna
    condiciones = {
        'BOGO': 'BOGOTA',
        'GOTA': 'BOGOTA',
        'LOCALID': 'BOGOTA',
        'MEDEL': 'MEDELLIN',
        'COMUNA': 'MEDELLIN',
        'POBLAD': 'MEDELLIN',
        'CALIFORNIA': 'SACRAMENTO',
        'MEXICALI': 'MEXICALI',
        'CALI': 'CALI',
        'DUPAR': 'VALLEDUPAR',
        'VALLE': 'CALI',
        'CARTAGE': 'CARTAGENA',
        'QUILLA': 'BARRANQUILLA',
        'BARRANQ': 'BARRANQUILLA',
        'VILLAVI': 'VILLAVICENCIO',
        'YOPAL': 'YOPAL',
        'YPAL': 'YOPAL',
        'TUNJA': 'TUNJA',
        'CUCUT': 'CUCUTA',
        'MANIZA': 'MANIZALES',
        'IBAG': 'IBAGUE',
        'NEIV': 'NEIVA', 
        'MARTA': 'SANTA MARTA',
        'BUCARA': 'BUCARAMANGA',
        'MONTERI': 'MONTERIA',
        'POPAYA': 'POPAYAN',
        'ARMENI': 'ARMENIA',
        'PEREIR': 'PEREIRA',
        'UBATE': 'UBATE',
        'CHIA': 'CHIA',
        'SOLEDA': 'SOLEDAD',
        'BELLO': 'BELLO',
        'SOACH': 'SOACHA',
        'BUGA': 'BUGA',
        'PAST': 'PASTO',
        'MANGA': 'BUCARAMANGA',
        'SOGAMO': 'SOGAMOSO',
        'DUITA': 'DUITAMA',
        # Puedes agregar más condiciones aquí según sea necesario
    }
    
    for condicion, ciudad in condiciones.items():
        if condicion in row:
            return ciudad
    
    return row


# Elimina la palabra que pertenezca a la lista
def eliminar_departamento(row):
    regiones = ['AMAZONAS','ANTIOQUIA', 'ARAUCA','ATLANTICO','BOLIVAR','BOYACA','CALDAS','CAQUETA',
                'CASANARE','CAUCA','CESAR','CHOCO','CUNDINAMARCA','CORDOBA','GUAINIA','GUAVIARE','HUILA',
                'LA GUAJIRA','MAGDALENA','META','NARINO','NORTE DE SANTANDER','PUTUMAYO','QUINDIO',
                'RISARALDA','ISLA','ISLAS','SANTANDER','SUCRE','TOLIMA','VALLE DEL CAUCA','VAUPES','VICHADA']

    # Crear un diccionario donde las claves sean los valores de la lista y los valores sean None
    dic= {region: '' for region in regiones}
    
    for condicion, ciudad in dic.items():
        if condicion in row:
            row = row.replace(condicion, ciudad).strip()
    
    return row


# Reemplaza algunas versiones adicionales de ciudad
def replace_cities_aliases(ciudad):
    bogota_aliases = ['BOTOTA','BTA','BOGTA','BOG','BOOGTA','BGTA','KENEDY','BGA','CUNDINAMARCA','BGOOTA','BOOTA','BOFOTA','BOGITA','CEDRITOS','BOBOTA','NOGOTA','BOGATA','BOHOTA','BOGPTA','CIUDAD VERDE','CIUDADVERDE','TINTAL','HAYUELOS','SALITRE']
    barranquilla_aliases = ['ATLANTICO','BAQ','BQLLLA','BQ','BQA','BQLLA','BARRAQNUILLA','BARRANUILLA','BQUILLLA']
    cali_aliases = ['VALLE DEL CAUCA']
    cartagena_aliases = ['CTG']
    ibague_aliases = ['IABGUE']
    medellin_aliases = ['ANTIOQUIA','POBLADO','MED','MDELLIN','RIO NEGRO']
    santanderdequilichao = ['DE QUILICHAO']
    if ciudad in bogota_aliases:
        return 'BOGOTA'
    elif ciudad in barranquilla_aliases:
        return 'BARRANQUILLA'
    elif ciudad in cali_aliases:
        return 'CALI'
    elif ciudad in cartagena_aliases:
        return 'CARTAGENA'
    elif ciudad in medellin_aliases:
        return 'MEDELLIN'
    elif ciudad in ibague_aliases:
        return 'IBAGUE'
    elif ciudad in santanderdequilichao:
        return 'SANTANDER DE QUILICHAO'
    else:
        return ciudad

In [ ]:
####  LECTURA DE CSV's REQUERIDOS  ###

inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Modelos_analitica/Maestras_municipios/"

## Codigos postales
codpost = spark.read.csv(inpath + 'codpost.csv', sep = '|', header=True, inferSchema=True)
# print(codpost.count())
codpost = codpost.toPandas()
#print(codpost.head(5))


## maestra de municipios de Colombia
dim_towns = spark.read.csv(inpath + 'dim_municipios.csv', sep = '|', header=True, inferSchema=True)
# print(dim_towns.count())
dim_towns = dim_towns.toPandas()
#print(dim_towns.head(5))


## localidades de Bogota
loc_bog = spark.read.csv(inpath + 'localidades_bogota.csv', sep = '|', header=True, inferSchema=True)
# print(loc_bog.count())
loc_bog = loc_bog.toPandas()
#print(loc_bog.head(5))


## Maestra Metodos pago
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Softix/Maestras/BillingCCName.csv'
MetodoPago = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')
#display(MetodoPago)
MetodoPago.createOrReplaceTempView("MetodoPago_vw")

## Maestra Segmentos
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Softix/Maestras/Btype.txt"
Btypes = spark.read.csv(inpath,header=True, inferSchema=True, sep=',')
#display(Btypes)
Btypes.createOrReplaceTempView("Btypes_vw")

## Maestra Segmentos
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Softix/Maestras/updated_venues_softix.csv"
venues_city= spark.read.csv(inpath,header=True, inferSchema=True, sep=',')
#display(Btypes)
venues_city.createOrReplaceTempView("venues_city_vw")


## Maestra aforos
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Softix/Maestras/softix_aforos.csv"
softix_aforos= spark.read.csv(inpath,header=True, inferSchema=True, sep=',')
#display(Btypes)
softix_aforos.createOrReplaceTempView("softix_aforos_vw")

In [ ]:
# AJUSTE MAESTRA AFOROS

# from pyspark.sql.functions import max,sum

# softix_aforos = softix_aforos.groupBy("Codigo_Evento",'Aforo').sum('Tickets').orderBy(col("Aforo").desc()).\
#     withColumn("Aforo", col("Aforo").cast("integer"))
# #print(tmp.count())
# #display(tmp)


# softix_aforos = softix_aforos.groupBy("Codigo_Evento").agg(max('Aforo').alias('Aforo'), sum('sum(Tickets)').alias('Tickets'))
# print(softix_aforos.count())

# # Escribimos el csv
# inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Softix/Maestras/softix_aforos/"

# write_table = softix_aforos.coalesce(1) # coalesce indica que la tabla se va a entregar en una sola particion
# write_table.write.csv(inpath, header=True, mode="overwrite", sep=',')

# #cambio de nombre
# files=dbutils.fs.ls(inpath)
# output_file= [x for x in files if x.name.startswith("part-")]

# # EN la misma carpeta de origen
# dbutils.fs.mv(output_file[0].path, f"{inpath}/softix_aforos.csv")

In [ ]:
Softix_Accounts ="dbo.Softix_Accounts"
Softix_Areas ="dbo.Softix_Areas"
Softix_Btypes ="dbo.Softix_Btypes"
Softix_Chains ="dbo.Softix_Chains"
Softix_Mtypes ="dbo.Softix_Mtypes"
Softix_Perfs ="dbo.Softix_Perfs"
Softix_Proms ="dbo.Softix_Proms"
Softix_Sellers ="dbo.Softix_Sellers"
Softix_Shows ="dbo.Softix_Shows"
Softix_Tixs ="dbo.Softix_Tixs"
Softix_Tnums ="dbo.Softix_Tnums"
Softix_Tnums_2014 = 'Softix.Tnums'
Softix_Venues ="dbo.Softix_Venues"

Softix_Accounts = spark.read.jdbc(url=jdbc_url, table =Softix_Accounts, properties=jdbc_properties)
#print(Softix_Accounts.count())
#Softix_Areas = spark.read.jdbc(url=jdbc_url, table =Softix_Areas, properties=jdbc_properties)
#print(Softix_Areas.count())
#Softix_Btypes = spark.read.jdbc(url=jdbc_url, table =Softix_Btypes, properties=jdbc_properties)
#print(Softix_Btypes.count())
#Softix_Chains = spark.read.jdbc(url=jdbc_url, table =Softix_Chains, properties=jdbc_properties)
#print(Softix_Chains.count())
Softix_Mtypes = spark.read.jdbc(url=jdbc_url, table =Softix_Mtypes, properties=jdbc_properties)
#print(Softix_Mtypes.count())
Softix_Perfs = spark.read.jdbc(url=jdbc_url, table =Softix_Perfs, properties=jdbc_properties)
#print(Softix_Perfs.count())
#Softix_Perfs.createOrReplaceTempView("Softix_Perfs_vw")
Softix_Proms = spark.read.jdbc(url=jdbc_url, table =Softix_Proms, properties=jdbc_properties)
#print(Softix_Proms.count())
#Softix_Sellers = spark.read.jdbc(url=jdbc_url, table =Softix_Sellers, properties=jdbc_properties)
#print(Softix_Sellers.count())
Softix_Shows = spark.read.jdbc(url=jdbc_url, table =Softix_Shows, properties=jdbc_properties)
#print(Softix_Shows.count())
#Softix_Tixs = spark.read.jdbc(url=jdbc_url, table =Softix_Tixs, properties=jdbc_properties)
#print(Softix_Tixs.count())
Softix_Tnums = spark.read.jdbc(url=jdbc_url, table =Softix_Tnums, properties=jdbc_properties)
Softix_Tnums_2014 = spark.read.jdbc(url=jdbc_url, table =Softix_Tnums_2014, properties=jdbc_properties)
#print(Softix_Tnums.count())
#Softix_Tnums_2014.createOrReplaceTempView("Softix_Tnums_2014_vw")
Softix_Venues = spark.read.jdbc(url=jdbc_url, table =Softix_Venues, properties=jdbc_properties)
#print(Softix_Venues.count())


### RENOMBRE DE CAMPOS

# Renombrar los campos quitando el prefijo 'Tnums.'
for col_name in Softix_Tnums.columns:
    new_col_name = col_name.replace("Tnums.", "")
    Softix_Tnums = Softix_Tnums.withColumnRenamed(col_name, new_col_name)

# Renombrar los campos quitando el prefijo 'Tnums.'
for col_name in Softix_Tnums_2014.columns:
    new_col_name = col_name.replace("Tnums.", "")
    Softix_Tnums_2014 = Softix_Tnums_2014.withColumnRenamed(col_name, new_col_name)

# Renombrar los campos quitando el prefijo 'Perfs.'
for col_name in Softix_Perfs.columns:
    new_col_name = col_name.replace("Perfs.", "")
    Softix_Perfs = Softix_Perfs.withColumnRenamed(col_name, new_col_name)

# Renombrar los campos quitando el prefijo 'Venues.'
for col_name in Softix_Venues.columns:
    new_col_name = col_name.replace("Venues.", "")
    Softix_Venues = Softix_Venues.withColumnRenamed(col_name, new_col_name)

# Renombrar los campos quitando el prefijo 'Accounts.'
for col_name in Softix_Accounts.columns:
    new_col_name = col_name.replace("Accounts.", "")
    Softix_Accounts = Softix_Accounts.withColumnRenamed(col_name, new_col_name)

# Renombrar los campos quitando el prefijo 'Proms.'
for col_name in Softix_Proms.columns:
    new_col_name = col_name.replace("Proms.", "")
    Softix_Proms = Softix_Proms.withColumnRenamed(col_name, new_col_name)

# Renombrar los campos quitando el prefijo 'Proms.'
for col_name in Softix_Shows.columns:
    new_col_name = col_name.replace("Shows.", "")
    Softix_Shows = Softix_Shows.withColumnRenamed(col_name, new_col_name)

display(Softix_Tnums.limit(10))
#print(Softix_Accounts.count())

id,Master,Tnum,DateTime,Channel,Seller,SellerID,Place,PlaceRegion,Prom,PromRegion,Chain,HowPaid,Action,Sign,Reason,ReasonNum,Active,EACustomer,Ecanon,Evname,Secnam,RZ,Pcat,Showtag,Seqnum,Paxnum,Paid,GrossPaid,Sales,GrossSales,Distributed,Bfee,BfeeSupp,BfeeBaseLine,Hfee,GST,GSTSiteLiable,Season,Account,J_AN_PREV_MJT,J_AN_NEXT_MJT,Returned,AllReturned,AdmitCnt,AdmitBfee,AdmitGST,AdmitPrice,AdmitPtype,AdmitQty,AdmitWas,Btype,BtypeName,Qty,Subs,BRCount,BillingAmount,BillingAuth,BillingAuthID,BillingAuthReqd,BillingBRName,BillingBRPhone,BillingCCName,BillingCCNum,BillingDateTime,BillingFundsMoved,BillingHowPaid,BillingInstallments,BillingInvMJT,BillingLastMJT,BillingMerchant,BillingMerchantName,BillingMJT,BilingPaytype,BillingSeller,BillingXPRY,SaleDtype,SaleDtypeNow
834921,"121003,5753",5753,121003 1838,Q,EXPAN1,,EXPAN,OCC,ADP,OCC,XI,1,BUY,1,,0,1,0,EADP2012891,EADP_3OCT2012,SORIITAL,RGA/A518-,2,OPASTOCAR2B,2926,2926,10000,10000,10000,10000,0,850,850,850,0,16,117,,0,,,,0,1,850,1262,9150,A,1,-,13,PFUTBOL,1,0,1,50000,OK,0,0,,,Cash,CASH,03-Oct-12 18:38:58.782,0,1,1,"121003,5752","121003,5753",,MIXTOS,"121003,5751",CASH,EXPAN1,,,
834922,"120918,5942",5942,120918 2113,W,QTROG1,QTROG1,QTROG,COL,CAD,ANT,CO,2,BUY,1,,0,1,0,ECAD2012877,ECAD_21SEP2012,SPRIFIRICIA,RA/A29-28,1,ABEBER12MD,47,47,115500,115500,115500,115500,0,8000,8000,8000,7500,16,2138,int,482163,"120423,6705","120925,526",,0,1,4000,6897,50000,A,2,-,2,TEATRO,2,0,1,100000,545367,0,1,JUAI FILIPI BIDOYA A,71791051,Visa,xxxxxxxxxxxx4970,18-Sep-12 21:13:57.538,0,2,1,"120918,5942","120918,5942",11FEE,INTERNET,"120918,5941",VISA,QTROG1,1013,V,V
834923,"121003,5755",5755,121003 1839,Q,EXIPA5,,EXIPA,OCC,ADP,OCC,XI,1,BUY,1,,0,1,0,EADP2012891,EADP_3OCT2012,SSUR,RGA/A1696-1697,4,OPASTOCAR2B,2927,2927,10000,10000,10000,10000,0,850,850,850,0,16,117,,0,,,,0,1,425,631,4575,A,2,-,13,PFUTBOL,2,0,1,10000,OK,0,0,,,Cash,CAS,03-Oct-12 18:39:03.133,0,1,1,"121003,5755","121003,5755",,MIXTOS,"121003,5754",CASH,EXIPA5,,,
834924,"120918,5944",5944,120918 2115,W,QTROG1,QTROG1,QTROG,COL,ICE,COL,CO,2,BUY,1,,0,1,1,EICE2012967,EICE_15DEC2012,SPLUSITIC,R11/B12-13,2,BFEDERER,10252,10252,1617500,1617500,1617500,1617500,0,30000,30000,30000,7500,16,5172,int,535403,,,,0,1,15000,108966,790000,B,2,-,21,DEPORTES,2,0,1,1580000,545449,0,1,CISAR RODRIGUI H,79156671,Amex,xxxxxxxxxxx3971,18-Sep-12 21:15:19.638,0,2,12,"120918,5944","120918,5944",11FEE,INTERNET,"120918,5943",AMEX,QTROG1,0215,P,P
834925,"121003,5760",5760,121003 1839,Q,EXIPA5,,EXIPA,OCC,ADP,OCC,XI,1,BUY,1,,0,1,0,EADP2012891,EADP_3OCT2012,SSUR,RGA/A1698-1699,4,OPASTOCAR2B,2929,2929,10000,10000,10000,10000,0,850,850,850,0,16,117,,0,,,,0,1,425,631,4575,A,2,-,13,PFUTBOL,2,0,1,10000,OK,0,0,,,Cash,CAS,03-Oct-12 18:39:25.596,0,1,1,"121003,5760","121003,5760",,MIXTOS,"121003,5759",CASH,EXIPA5,,,
834926,"120918,5946",5946,120918 2116,W,QTROG1,QTROG1,QTROG,COL,FCC,COL,CO,2,BUY,1,,0,1,0,EFCC2012883O,EFCC_27SEP2012O,SPLATIAIIQUI,RC/F9-12,2,BFIGARO12,304,304,463500,463500,463500,463500,0,24000,0,24000,7500,16,4345,int,144207,"120413,5154","181001,75",,0,1,6000,14897,108000,F,4,-,1,CULTURAL,4,0,1,432000,908285,0,1,Andres I Pacheco Rincon,51918767,Master Card,xxxxxxxxxxxx1912,18-Sep-12 21:16:02.950,0,2,24,"120918,5946","120918,5946",22FEE,INTERNET,"120918,5945",MACH,QTROG1,0718,P,P
834927,"121003,5762",5762,121003 1839,Q,EXPAN1,,EXPAN,OCC,ADP,OCC,XI,1,BUY,1,,0,1,0,EADP2012891,EADP_3OCT2012,SSUR,RGA/A1700-,4,OPASTOCAR2B,2931,2931,5000,5000,5000,5000,0,425,425,425,0,16,59,,0,,,,0,1,425,631,4575,A,1,-,13,PFUTBOL,1,0,1,5000,OK,0,0,,,Cash,CAS,03-Oct-12 18:39:28.246,0,1,1,"121003,5762","121003,5762",,MIXTOS,"121003,5761",CASH,EXPAN1,,,
834928,"120918,5948",5948,120918 2119,W,QTROG1,QTROG1,QTROG,COL,TAG,CEN,CO,2,BUY,1,,0,1,1,ETAG2012915,ETAG_27OCT2012,SVIP,RGA/P3-5,2,CPARRANDON,11,11,301500,301500,301500,301500,0,24000,24000,24000,7500,16,4345,int,345695,"120212,37","170810,17820",,0,1,8000,12414,90000,P,3,-,7,CONCIE,3,0,1,270000,545679,0,1,jhon sanchez uribe,79360850,Amex,xxxxxxxxx

In [ ]:
#print(Softix_Tnums.count())

# tmp = Softix_Tnums.select('Prom').distinct()
# display(tmp)

5813685


In [ ]:
#display(Softix_Shows.limit(10))

# tmp = Softix_Shows.filter(col('Code') == 'ANGRA')#.limit(100)
# display(tmp)

In [ ]:
# tmp = Softix_Tnums.select('SaleDtypeNow').distinct()
# display(tmp)

In [ ]:
# Softix_Tnums = Softix_Tnums.\
#     withColumn("Fecha_compra", to_timestamp("DateTime", "yyMMdd HHmm")).\
#     withColumn("Anio_compra", year("Fecha_compra"))
# # #.select('Evname','BillingDateTime')

# Softix_Tnums_2014 = Softix_Tnums_2014.\
#     withColumn("Fecha_compra", to_timestamp("DateTime", "yyMMdd HHmm")).\
#     withColumn("Anio_compra", year("Fecha_compra"))

In [ ]:
#### TRANSFORMACIONES ####

# variables a concatenar
id_cliente_Tnums = ["Season",'Account']
id_cliente_Accounts = ["Season",'Number']
id_expediente = ["Id_cliente",'DateTime']

Softix_Tnums = Softix_Tnums.\
    filter((col('AdmitPtype') != 'C') & 
           (col('Action') != 'COM') &
           (col('Prom') != 'CTK')).\
    withColumn("Id_cliente",concat_ws("|", *id_cliente_Tnums)).\
    withColumn("FILE_NUMBER",concat_ws("|", *id_expediente)).\
    withColumn("Fecha_compra", to_timestamp("DateTime", "yyMMdd HHmm")).\
    withColumn("Anio_compra", year("Fecha_compra")).\
    filter(col('Anio_compra') != '2014')
    #withColumn("Fecha_compra", to_timestamp("BillingDateTime", "dd-MMM-yy HH:mm:ss.SSS"))
#.select('Evname','BillingDateTime')


Softix_Tnums_2014 = Softix_Tnums_2014.\
    filter((col('AdmitPtype') != 'C') & 
           (col('Action') != 'COM') &
           (col('Prom') != 'CTK')).\
    withColumn("Id_cliente",concat_ws("|", *id_cliente_Tnums)).\
    withColumn("FILE_NUMBER",concat_ws("|", *id_expediente)).\
    withColumn("Fecha_compra", to_timestamp("DateTime", "yyMMdd HHmm")).\
    withColumn("Anio_compra", year("Fecha_compra")).\
    filter(col('Anio_compra') == '2014')

# Union del Tnums
Softix_Tnums = Softix_Tnums.union(Softix_Tnums_2014)

####
Softix_Perfs = Softix_Perfs.select(*[col for col in Softix_Perfs.columns if col != 'id']).distinct().\
    withColumn("Fecha_evento", to_timestamp("EndTime", "yyMMdd HHmm"))


Softix_Proms = Softix_Proms.select(*[col for col in Softix_Proms.columns if col not in ['id','Num']]).\
    withColumnRenamed('Desc', 'Descripcion').distinct()


Softix_Accounts = Softix_Accounts.select(*[col for col in Softix_Accounts.columns if col not in ['id']]).\
    withColumn("Id_cliente",concat_ws("|", *id_cliente_Accounts)).distinct()
    

Softix_Venues = Softix_Venues.withColumnRenamed('Desc', 'Descripcion').distinct()

Softix_Shows = Softix_Shows.withColumnRenamed('Desc', 'Descripcion').distinct()


In [ ]:
# # # tmp = Softix_Tnums.groupby(year("Fecha_compra")).count().reset_index(name='count')
# # # display(tmp)

# from pyspark.sql.functions import sum

# tmp = softix_aforos.groupBy("Anio_compra",'Aforo').sum().orderBy(col("count").desc())#.\
# #print(tmp.count())
# #display(tmp)

In [ ]:
#display(Softix_Tnums.limit(10))

In [ ]:
# tmp = Softix_Tnums.select('Evname').distinct()
# #display(tmp)
# print(tmp.count()) 

In [ ]:
# tmp = Softix_Tnums.groupBy("FILE_NUMBER").count().orderBy(col("count").desc())#.\
# #print(tmp.count())
# display(tmp.limit(1000))

In [ ]:
tmp = Softix_Tnums.groupBy("Anio_compra").count().orderBy(col("count").desc())#.\
#print(tmp.count())
display(tmp)

Anio_compra,count
2014,1056991
2016,707494
2013,672307
2011,617012
2004,337386
2015,310901
2019,281587
2010,281438
2005,279076
2017,277476


In [ ]:
# from pyspark.sql.functions import max

# tmp = Softix_Tnums.filter(col('Anio_compra') == '2014')#.select(max('Fecha_compra'))
# print(tmp.count())
# #display(tmp)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-2817531399402343>, line 4
      1 from pyspark.sql.functions import max
      3 tmp = Softix_Tnums.filter(col('Anio_compra') == '2014')#.select(max('Fecha_compra'))
----> 4 print(tmp.count())
      5 #display(tmp)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:1206, in DataFrame.count(self)
   1183 def count(self) -> int:
   1184     """Returns the number of rows in this :class:`DataFrame`.
   1185 
   1186     .. versionadded:: 1.3.0
   (...)
   1

In [ ]:
# tmp = Softix_Accounts.filter(col('Id_cliente') == '|0')
# #print(tmp.count())
# display(tmp)

In [ ]:
# print(Softix_Accounts.count())
dirty_cities = Softix_Accounts.groupBy("City").count().orderBy(col("count").desc())#.\
print(dirty_cities.count())
dirty_cities = dirty_cities.toPandas()

36441


In [ ]:
# #tmp = Softix_Accounts.select('City').distinct().orderBy(col("City").desc())
# tmp = Softix_Accounts.groupBy("City").count().orderBy(col("count").desc()).filter(col('count') >= 100)#.\
# print(tmp.count())
# display(tmp)

In [ ]:
## Transformacion 1: Codigos postales

# left join de dirty_cities con codigo postal
cities_ = pd.merge(dirty_cities, codpost[['Codigo Postal', 'Ciudad']],\
                          left_on='City', right_on= codpost['Codigo Postal'].astype(str),\
                          how='left')

# Rellenar los valores nulos en 'Ciudad' con los valores de 'MAIN_ADDR_TOWN'
cities_['Ciudad'].fillna(cities_['City'], inplace=True)

print(cities_.count())
print(cities_['Ciudad'].unique())

City             36441
count            36441
Codigo Postal        3
Ciudad           36441
dtype: int64
['' 'BTA' 'BOGOTA BOGOT?DC' ... 'LEIPZIG SACHSEN' 'DAFSDFSADF'
 'MEDE POBLADO']


In [ ]:
# # Filtrar el DataFrame
# resultado_filtrado = cities_[cities_['Codigo Postal'].notnull()]
# print(resultado_filtrado.head(5))

In [ ]:
## Transformacion 2: limpieza de texto y join con localidades bogota

total = sum(cities_['count'])
#print('num registros: ', total)

# Limpieza del campo ciudad de acuerdo con la funcion limpiar_texto
dim_municipios_ = cities_.sort_values(by='count', ascending=False).\
       assign(Ciudad = lambda x: x['Ciudad'].apply(limpiar_texto),
              PARTICIPACION = lambda x: x['count']/total,
              TOTAL = lambda x: np.cumsum(x['PARTICIPACION'])).\
       rename(columns = {'count': 'COUNT'})
       #groupby('DEPARTAMENTO').agg({'COUNT': 'sum','PORCENTAJE': 'sum'}).reset_index().\


# left join con localidades
dim_municipios_1 = pd.merge(dim_municipios_, loc_bog,\
                          left_on='Ciudad', right_on= 'LOCALIDAD',\
                          how='left')

# Rellenar los valores nulos en 'CIUDAD' (de loc_bog)con los valores de 'Ciudad'
dim_municipios_1 = dim_municipios_1.assign(CIUDAD = lambda x: x['CIUDAD'].fillna(dim_municipios_1['Ciudad'])).\
                                    assign(CIUDAD = lambda x: x['CIUDAD'].apply(asignar_ciudad)).\
                                    assign(CIUDAD = lambda x: x['CIUDAD'].apply(replace_cities_aliases))

#print(dim_municipios_1.head(6))

# tmp = dim_municipios_1.groupby("CIUDAD").agg({'COUNT': 'sum'})\
#     .sort_values(by='COUNT', ascending=False)

# print(tmp.head(15))

In [ ]:
#tmp = dim_municipios_1.groupby(year("CIUDAD")).size().reset_index(name='count').\


In [ ]:
## Transformacion 3:  llave departamento para traer capital a los que tienen departamento en el campo ciudad
dim_regiones = dim_towns[['DEPARTAMENTO','CAPITAL']].drop_duplicates()
#print(dim_regiones)

dim_municipios = pd.merge(dim_municipios_1, dim_regiones,\
                          left_on='CIUDAD', right_on= 'DEPARTAMENTO',\
                          how='left')

# Rellenar los valores nulos en 'Ciudad' con los valores de 'MAIN_ADDR_TOWN'
dim_municipios['CAPITAL'].fillna(dim_municipios['CIUDAD'], inplace=True)

# Finalmente seleccionamos los campos necesarios
dim_municipios = dim_municipios[['City','COUNT','PARTICIPACION','CAPITAL']]

dim_municipios.rename(columns={'CAPITAL': 'CIUDAD'}, inplace=True)

# tmp = dim_municipios.groupby("CIUDAD").agg({'COUNT': 'sum'})\
#     .sort_values(by='COUNT', ascending=False)

# print(tmp.head(15))

In [ ]:
## Transformacion 4:  left join para validar cuales valores quedaron limpios y traer la region y el departamento
dim_municipios = pd.merge(dim_municipios, dim_towns,\
                          left_on='CIUDAD', right_on= 'MUNICIPIO',\
                          how='left')

# Traer los valores que ya sabiamos eran nulos o ''
dim_municipios['MUNICIPIO'] = np.where(
    (dim_municipios['CIUDAD'] == '') |
    (dim_municipios['CIUDAD'] == 'nan') |
    (dim_municipios['CIUDAD'] == 'None') |
    (dim_municipios['CIUDAD'].isnull()),
    'SIN DATO',
    dim_municipios['MUNICIPIO']
)
#print(dim_municipios.head(5))

# tmp = dim_municipios.groupby("MUNICIPIO").agg({'COUNT': 'sum'})\
#     .sort_values(by='COUNT', ascending=False)

# print(tmp.head(15))

In [ ]:
# Finalmente seleccionamos los campos necesarios
dim_municipios = dim_municipios[['City','COUNT','PARTICIPACION','CIUDAD','MUNICIPIO','REGION','DEPARTAMENTO','CAPITAL','PAIS']]

# Valores con MUNICIPIO LIMPIO
notnull = dim_municipios[dim_municipios['MUNICIPIO'].notnull()]

print('Registros: ', notnull['COUNT'].sum())
print('Percent: ', notnull['PARTICIPACION'].sum())

#################
# Descriptivos
# print(notnull.head(5))
# print(' ')
# print('Dimensiones:')
print(notnull.shape)


Registros:  2327842
Percent:  0.9523946290788926
(20289, 9)


In [ ]:
# Valores con MUNICIPIO sin limpiar
null = dim_municipios[dim_municipios['MUNICIPIO'].isnull()]

print('Registros: ', null['COUNT'].sum())
print('Percent: ', null['PARTICIPACION'].sum())

#################
# Descriptivos
print('Dimensiones:')
print(null.shape)

Registros:  116357
Percent:  0.04760537092110749
Dimensiones:
(16152, 9)


In [ ]:
# Aplicamos la funcion de eliminar departamento para limpiar valores estilo: 'ABEJORRAL ANTIOQUIA'
null = null[['City','COUNT','PARTICIPACION','CIUDAD']].\
    assign(CIUDAD = lambda x: x['CIUDAD'].apply(eliminar_departamento))


# left join para traer la region y el departamento NUEVAMENTE
null = pd.merge(null, dim_towns,\
                    left_on='CIUDAD', right_on= 'MUNICIPIO',\
                    how='left')

null2 = null[null['MUNICIPIO'].notnull()]

print('Registros: ', null2['COUNT'].sum())
print('Percent: ', null2['PARTICIPACION'].sum())

# #################
# # Descriptivos
# print('Dimensiones:')
# print(null2.shape)

Registros:  44614
Percent:  0.01825301458678283


In [ ]:
# UNIMOS AMBAS TABLAS PARA TENER LA LIMPIEZA COMPLETA
dim_municipios_ = pd.concat([notnull, null2])

print('Registros: ', dim_municipios_['COUNT'].sum())
print('Percent: ', dim_municipios_['PARTICIPACION'].sum())
# Rellenar los valores nulos en 'Ciudad' con los valores de 'MAIN_ADDR_TOWN'
#dim_municipios_['Ciudad'].fillna(cities_['MAIN_ADDR_TOWN'], inplace=True)

# tmp = dim_municipios.groupby("MUNICIPIO").agg({'COUNT': 'sum'})\
#     .sort_values(by='COUNT', ascending=False)

# print(tmp.head(15))

# Convertir el DataFrame de Pandas a un DataFrame de PySpark
dim_municipios_ = spark.createDataFrame(dim_municipios_)
dim_municipios_.createOrReplaceTempView("dim_municipios_vw")

#display(dim_municipios_)

Registros:  2372456
Percent:  0.9706476436656755


In [ ]:
# inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Modelos_analitica/Maestras_municipios/"

# write_table = dim_municipios_.coalesce(1) # coalesce indica que la tabla se va a entregar en una sola particion
# write_table.write.csv(inpath + "dirty_cities", header=True, mode="overwrite", sep='|')

# #cambio de nombre
# files=dbutils.fs.ls(inpath + 'dirty_cities/')
# output_file= [x for x in files if x.name.startswith("part-")]

# # EN la misma carpeta de origen
# dbutils.fs.mv(output_file[0].path, f"{inpath + 'dirty_cities/'}/dirty_cities.csv")

In [ ]:
# #print(Softix_Tnums.count())

# tmp = Softix_Tnums.filter(col('Account') == '650292')#.limit(100)
# display(tmp)

In [ ]:
# tmp = Softix_Accounts.filter(col('Number') == '650292')#.limit(100)
# display(tmp)


#display(Softix_Mtypes.limit(50)) #650292

In [ ]:
# # PRUEBA
# # Convertir la cadena de fecha al formato de fecha correcto
# tmp = Softix_Tnums.select('Evname','BillingDateTime').\
#     withColumn("Fecha_compra", to_timestamp("BillingDateTime", "dd-MMM-yy HH:mm:ss.SSS")).\
#     withColumn("fecha_evento_string", regexp_extract("Evname", r'_(\d{1,2}\s?[A-Za-z]{3}\d{4})', 1))#.\
#     #withColumn("Fecha_evento", to_timestamp("fecha_evento_string", "dMMMyyyy"))

# display(tmp.limit(5))

In [ ]:
# from pyspark.sql import Window

# # Agregar una columna que indique si es duplicado o no
# window_spec = Window().partitionBy("Codigo_Evento").orderBy("Codigo_Evento")

# df_spark = softix_aforos.withColumn("es_duplicado", count("Codigo_Evento").over(window_spec) > 1)


# # Filtrar solo las filas duplicadas
# duplicados = df_spark.filter("es_duplicado")#.select('Evento').distinct()
# #duplicados = duplicados.withColumn('Evento_N', lit('Hola'))
# # DF = df3.toPandas()

# # # Verificar duplicados en la columna 'llave_identificadora'
# # duplicados = DF[DF.duplicated(subset=['Key_evento'], keep=False)]

# #df_spark = spark.createDataFrame(duplicados)
# # print(df.count())
# display(duplicados)

In [ ]:
# display(Softix_Tnums.limit(9))

In [ ]:
# tmp1 = Softix_Accounts.select('Id_cliente').distinct()
# # tmp2 = Softix_Proms.select('Code').distinct()

# # tmp3 = tmp1.alias("a").join(
# #     tmp2.alias("b"),
# #     col("a.Prom") == col("b.Code"),
# #     "left_outer"
# # )

# print(tmp1.count())
# #display(tmp1)

In [ ]:
# # # tmp = Softix_Perfs.filter(col('Code') == 'ESFIE')
# # # display(tmp)

# tmp = Softix_Accounts.select('City').distinct()
# #print(tmp.count())
# #print(Softix_Proms.count())
# display(tmp)

# # #display(Softix_Proms.limit(30))

In [ ]:
Softix_Accounts.createOrReplaceTempView("Softix_Accounts_vw")
#Softix_Areas.createOrReplaceTempView("Softix_Areas_vw")
#Softix_Btypes.createOrReplaceTempView("Softix_Btypes_vw")
#Softix_Chains.createOrReplaceTempView("Softix_Chains_vw")
Softix_Mtypes.createOrReplaceTempView("Softix_Mtypes_vw")
Softix_Perfs.createOrReplaceTempView("Softix_Perfs_vw")
Softix_Proms.createOrReplaceTempView("Softix_Proms_vw")
#Softix_Sellers.createOrReplaceTempView("Softix_Sellers_vw")
Softix_Shows.createOrReplaceTempView("Softix_Shows_vw")
#Softix_Tixs.createOrReplaceTempView("Softix_Tixs_vw")
Softix_Tnums.createOrReplaceTempView("Softix_Tnums_vw")
Softix_Venues.createOrReplaceTempView("Softix_Venues_vw")

In [ ]:
# tmp = spark.sql('''
# SELECT DISTINCT
#       Id_cliente,
#       LAST_VALUE(Name) OVER (PARTITION BY Id_cliente ORDER BY Id_cliente desc) AS Nombre_cliente,
#       Email,
#       HomePhone,
#       City,
#       Gender
#       FROM Softix_Accounts_vw
#       WHERE Id_cliente = 'tel|64752'
# ''')
# display(tmp.limit(100))

In [ ]:
# print(Softix_Perfs.count())

# softix = spark.sql('''
# SELECT
#       p.Code as Sesion,
#       s.Descripcion AS Evento,
#       p.Fecha_evento,
#       v.Descripcion AS Venue,
#       --p.Venue as Cod_Venue,
#       vc.Ciudad AS Ciudad_venue
#       FROM Softix_Perfs_vw p
#       LEFT JOIN Softix_Shows_vw s
#         ON (p.Show1 = s.Code)
#       LEFT JOIN Softix_Venues_vw v
#         ON (p.Venue = v.Code) 
#       LEFT JOIN venues_city_vw vc
#         ON (v.Descripcion = vc.Descripcion) ''')

# print(softix.count())
# #display(softix.limit(100))

In [ ]:
softix = spark.sql('''
    -- Ajuste de promotores con codigo repetido
    WITH perfs as (
      SELECT
      p.Code as Key_evento,
      s.Descripcion AS Evento,
      p.Fecha_evento,
      v.Descripcion AS Venue,
      --p.Venue as Cod_Venue,
      vc.Ciudad AS Ciudad_venue
      FROM Softix_Perfs_vw p
      LEFT JOIN Softix_Shows_vw s
        ON (p.Show1 = s.Code)
      LEFT JOIN Softix_Venues_vw v
        ON (p.Venue = v.Code) 
      LEFT JOIN venues_city_vw vc
        ON (v.Descripcion = vc.Descripcion)
    ),
    
    proms as (
      SELECT DISTINCT
      Code,
      FIRST_VALUE(Descripcion) OVER (PARTITION BY Code) AS Promotor
      FROM Softix_Proms_vw
    ),   

    -- Ajuste de accounts con codigo repetido
    accounts as (
      SELECT DISTINCT
      Id_cliente,
      LAST_VALUE(Name) OVER (PARTITION BY Id_cliente ORDER BY Id_cliente desc) AS Nombre_cliente,
      Email,
      HomePhone as Movil,
      CASE WHEN Address1 IS NULL
        THEN Address2
        ELSE Address1 END AS Direccion,
      b.MUNICIPIO as Ciudad_del_cliente,
      CountryName as Pais_cliente,
      Gender as Genero
      FROM Softix_Accounts_vw a
      LEFT JOIN dim_municipios_vw b
        ON (a.City = b.MUNICIPIO)
    )

    SELECT
    p.*,
    --s.BTypeName as Segmento,
    b.Segmento as Tema_gral,
    s.Secnam as Categoria_logica,
    pr.Promotor,
    s.FILE_NUMBER,
    s.Fecha_compra as Fecha_Compra,
    s.Id_cliente as Id_contacto,
    a.Nombre_cliente,
    a.Email,
    a.Movil,
    a.Direccion,
    a.Ciudad_del_cliente, -- otra limpieza enorme
    a.Pais_cliente,
    a.Genero,
    CASE WHEN s.Channel = 'Q' THEN 'Punto de venta'
         WHEN s.Channel = 'W' THEN 'Web'
         WHEN s.Channel = 'B' THEN 'Box Office/Taquilla'
         WHEN s.Channel = 'T' THEN 'Call center'
      ELSE 'Punto de venta' END AS Subcanal,
    m.Metodo_pago AS Metodo_de_pago,
    --s.BillingCCName as Metodo_pago,
    s.SaleDtypeNow as Metodo_entrega,
    '' as Familia_producto,
    sa.Aforo,
    sa.Tickets as Tickets_sesion,
    SUM(s.Qty) as Tickets_comprados,
    SUM(s.Paid) AS Recaudo,
    SUM(s.Bfee) as Bfee,
    SUM(s.Hfee) AS Hfee
    FROM Softix_Tnums_vw s
    LEFT JOIN perfs p
      ON (s.Evname = p.Key_evento)
    LEFT JOIN accounts a
      ON (s.Id_cliente = a.Id_cliente)
    LEFT JOIN proms pr
      ON (s.Prom = pr.Code)
    LEFT JOIN Btypes_vw b
      ON (s.BTypeName = b.BtypeName)
    LEFT JOIN MetodoPago_vw m
      ON (s.BillingCCName = m.BillingCCName)
    LEFT JOIN softix_aforos_vw sa
      ON (s.Evname = sa.Codigo_Evento)  
    GROUP BY
    p.Key_evento,p.Evento,p.Fecha_evento,p.Venue,p.Ciudad_venue,b.Segmento,s.Secnam,s.Secnam,pr.Promotor,s.FILE_NUMBER,s.Fecha_compra,s.Id_cliente,a.Nombre_cliente,a.Email,a.Movil,a.Direccion,a.Ciudad_del_cliente,a.Pais_cliente,a.Genero,s.Channel,m.Metodo_pago,s.SaleDtypeNow,sa.Aforo,sa.Tickets
''')


#print(softix.count())
#display(softix.limit(100))


In [ ]:
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Audiencias_mercadeo/Ds_G_audiencias_mercadeo.parquet"
audiencias = spark.read.parquet(inpath)

In [ ]:
# tmp = audiencias.filter(col('Familia_producto') != 'Pase de visita','COLOMBIA')
# #display(tmp.limit(100))

# tmp = tmp.groupBy("Anio_compra").count().orderBy(col("count").desc())#.\

In [ ]:
columnas_softix = ['Key_evento',"Evento","Fecha_evento",'Venue',"Ciudad_venue","Tema_gral","Promotor",'FILE_NUMBER',"Fecha_Compra",'Id_contacto',"Ciudad_del_cliente","Subcanal","Metodo_de_pago","Tickets_comprados","Recaudo",'Familia_producto','Aforo','Tickets_sesion']

columnas_secutix = ['Key_evento',"Evento","Fecha_evento",'Venue',"Ciudad_venue","Tema_gral","Promotor",'FILE_NUMBER',"Fecha_Compra",'Id_contacto',"Ciudad_del_cliente","Subcanal","Medio de Pago General","Tickets_comprados","Recaudo",'Familia_producto','Aforo']
 
### A) seleccion de columnas ###
softix_ = softix.select(columnas_softix)
audiencias = audiencias.select(columnas_secutix).\
    withColumnRenamed('Metodo_de_pago', 'Medio de Pago General').\
    withColumn("Tickets_sesion", lit(0))

tuboleta = softix_.union(audiencias)

In [ ]:
# # print(Softix_Accounts.count())
# from pyspark.sql.functions import sum

# tmp = tuboleta.filter(col('Tema_gral').like('%oncierto%')).groupBy("Evento").sum('Tickets_comprados').alias('Tickets_comprados').orderBy(col("sum(Tickets_comprados)").desc())#.\
# #print(dirty_cities.count())
# display(tmp.limit(50))

In [ ]:
# Impresion
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Softix/Historico/"
join_df_coalesced = tuboleta.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_G_Softix.parquet")

True

In [ ]:
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Softix/Historico/Ds_G_Softix.parquet"
softix = spark.read.parquet(inpath)

In [ ]:
# tmp = softix.groupBy("Anio_compra").agg(max('Aforo').alias('Aforo'), sum('sum(Tickets)').alias('Tickets'))
# # print(softix_aforos.count())

# Softix_Tnums = Softix_Tnums.\
#     withColumn("Id_cliente",concat_ws("|", *id_cliente_Tnums)).\
#     withColumn("FILE_NUMBER",concat_ws("|", *id_expediente)).\
#     withColumn("Fecha_compra", to_timestamp("DateTime", "yyMMdd HHmm")).\
#     withColumn("Anio_compra", year("Fecha_compra")).\
#     filter(col('Anio_compra') != '2014')

# Softix_Tnums.createOrReplaceTempView("Softix_Tnums_vw")

tmp = spark.sql('''
    WITH tmp as (
        SELECT DISTINCT
        Key_evento,
        Anio_compra,
        Mes_compra,
        Tickets_sesion
        from softix_vw
    ),

    tmp2 as (
        SELECT
        Anio_compra,
        Mes_compra,
        sum(Tickets_sesion) as Tickets_sesion
        from tmp
        group by Anio_compra,Mes_compra
    )          
    
      SELECT
      s.Anio_compra,
      s.Mes_compra,
      sum(s.Tickets_comprados),
      t2.Tickets_sesion
      FROM softix_vw s
      LEFT JOIN tmp2 t2
        ON (s.Anio_compra = t2.Anio_compra AND s.Mes_compra = t2.Mes_compra)
      GROUP BY s.Anio_compra,s.Mes_compra,t2.Tickets_sesion
      ORDER BY s.Anio_compra,s.Mes_compra
''')

display(tmp)

Anio_compra,Mes_compra,sum(Tickets_comprados),Tickets_sesion
2012,1,324730.0,305419
2012,2,342029.0,512406
2012,3,279011.0,547971
2012,4,299097.0,490828
2012,5,139147.0,326545
2012,6,119899.0,454514
2012,7,231515.0,626114
2012,8,275664.0,680269
2012,9,254034.0,621678
2012,10,265655.0,595022


In [ ]:
# tmp = softix.groupBy("Anio_compra").agg(max('Aforo').alias('Aforo'), sum('sum(Tickets)').alias('Tickets'))
# # print(softix_aforos.count())

Softix_Tnums = Softix_Tnums.\
    withColumn("Id_cliente",concat_ws("|", *id_cliente_Tnums)).\
    withColumn("FILE_NUMBER",concat_ws("|", *id_expediente)).\
    withColumn("Fecha_compra", to_timestamp("DateTime", "yyMMdd HHmm")).\
    withColumn("Anio_compra", year("Fecha_compra")).\
    withColumn("Mes_compra", month("Fecha_compra"))

Softix_Tnums.createOrReplaceTempView("Softix_Tnums_vw")

tmp = spark.sql('''
    WITH tmp as (
        SELECT DISTINCT
        s.Evname,
        s.Anio_compra,
        s.Mes_compra,
        t2.Tickets as Tickets_sesion
        from Softix_Tnums_vw s
        LEFT JOIN softix_aforos_vw t2
          ON (s.Evname = t2.Codigo_Evento)
    ),

    tmp2 as (
        SELECT
        Anio_compra,
        Mes_compra,
        sum(Tickets_sesion) as Tickets_sesion
        from tmp
        group by Anio_compra,Mes_compra
    )          
    
      SELECT
      s.Anio_compra,
      s.Mes_compra,
      sum(s.Qty) as tickets_base,
      t2.Tickets_sesion as tickets_consolidado
      FROM Softix_Tnums_vw s
      LEFT JOIN tmp2 t2
        ON (s.Anio_compra = t2.Anio_compra AND s.Mes_compra = t2.Mes_compra)
      GROUP BY s.Anio_compra,s.Mes_compra,t2.Tickets_sesion
      ORDER BY s.Anio_compra,s.Mes_compra
''')


display(tmp)

Anio_compra,Mes_compra,tickets_base,tickets_consolidado
2012,1,571508,305961
2012,2,461044,518379
2012,3,436119,566140
2012,4,490325,497399
2012,5,264015,330980
2012,6,277300,455422
2012,7,387998,627326
2012,8,538850,680712
2012,9,459825,623185
2012,10,579344,595580


In [ ]:
# tmp = softix.groupBy("Anio_compra").agg(max('Aforo').alias('Aforo'), sum('sum(Tickets)').alias('Tickets'))
# # print(softix_aforos.count())

Softix_Tnums_2014 = Softix_Tnums_2014.\
    withColumn("Fecha_compra", to_timestamp("DateTime", "yyMMdd HHmm")).\
    withColumn("Anio_compra", year("Fecha_compra")).\
    withColumn("Mes_compra", month("Fecha_compra"))
#display(tmp)
#softix.createOrReplaceTempView("softix_vw")
Softix_Tnums_2014.createOrReplaceTempView("Softix_Tnums_2014_vw")

tmp = spark.sql('''
    WITH tmp as (
        SELECT DISTINCT
        s.Evname,
        s.Anio_compra,
        s.Mes_compra,
        t2.Tickets as Tickets_sesion
        from Softix_Tnums_2014_vw s
        LEFT JOIN softix_aforos_vw t2
          ON (s.Evname = t2.Codigo_Evento)
    ),

    tmp2 as (
        SELECT
        Anio_compra,
        Mes_compra,
        sum(Tickets_sesion) as Tickets_sesion
        from tmp
        group by Anio_compra,Mes_compra
    )          
    
      SELECT
      s.Anio_compra,
      s.Mes_compra,
      sum(s.Qty) as tickets_base,
      t2.Tickets_sesion as tickets_consolidado
      FROM Softix_Tnums_2014_vw s
      LEFT JOIN tmp2 t2
        ON (s.Anio_compra = t2.Anio_compra AND s.Mes_compra = t2.Mes_compra)
      GROUP BY s.Anio_compra,s.Mes_compra,t2.Tickets_sesion
      ORDER BY s.Anio_compra,s.Mes_compra
''')

display(tmp)

In [ ]:
from pyspark.sql.functions import sum

## Maestra aforos
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/TU_BOLETA/Softix/Consolidados/Inboleta_2014.txt"
softix_2014= spark.read.csv(inpath,header=True, inferSchema=True, sep='|')
#display(Btypes)
softix_2014.createOrReplaceTempView("softix_2014_vw")

id_cliente_Tnums = ["Season",'No_Cuenta']
id_cliente_Accounts = ["Season",'Number']
id_expediente = ["Id_cliente",'Fecha']

softix_2014 = softix_2014.\
    withColumn("Fecha_compra", to_timestamp("Fecha", "dd-MMM-yy")).\
    withColumn("Anio_compra", year("Fecha_compra")).\
    withColumn("Mes_compra", month("Fecha_compra"))

    
    # withColumn("Id_cliente",concat_ws("|", *id_cliente_Tnums)).\
    # withColumn("FILE_NUMBER",concat_ws("|", *id_expediente)).\
        
#display(softix_2014.limit(10))

tmp = softix_2014.groupBy("Anio_compra",'Mes_compra').agg(sum('Cantidad').alias('Aforo')).orderBy(col("Anio_compra").asc(),col("Mes_compra").asc())
display(tmp)

Anio_compra,Mes_compra,Aforo
2014,1,383596
2014,2,451991
2014,3,385106
2014,4,504091
2014,5,894665
2014,6,279476
2014,7,543156
2014,8,682624
2014,9,478835
2014,10,697437
